In [ ]:
!git clone https://github.com/kmkurn/pytorch-crf.git
!pip install ./pytorch-crf

In [ ]:
!pip install torch transformers datasets seqeval tqdm matplotlib
!pip install 'transformers[sentencepiece]'

In [ ]:

# Imports
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, DataCollatorForTokenClassification, TrainingArguments, get_scheduler
from datasets import load_dataset
from torchcrf import CRF
from seqeval.metrics import classification_report
from tqdm import tqdm
import os
from google.colab import drive
import gzip

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define the directory to save checkpoints in Google Drive
google_drive_dir = "/content/drive/My Drive/NER_checkpoints"
os.makedirs(google_drive_dir, exist_ok=True)

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# resume_from_checkpoint = "/content/drive/My Drive/NER_checkpoints/latest_checkpoint.pt.gz"
resume_from_checkpoint = None
start_epoch = 0

# Load the CoNLL-2003 dataset
dataset = load_dataset("MultiCoNER/multiconer_v2", "English (EN)")

# Reduce the dataset size (30% of the original size) for quicker experimentation
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(int(len(dataset["train"]))))
dataset["validation"] = dataset["validation"].shuffle(seed=42).select(range(int(len(dataset["validation"]))))
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(int(len(dataset["test"]))))

# Get label names
label_list = dataset["train"].features["ner_tags_index"].feature.names
num_labels = len(label_list)

# Load tokenizer
model_name = "bert-large-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization and label alignment function
def tokenize_and_align_labels_with_crf(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples["ner_tags_index"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_and_align_labels_with_crf, batched=True)

# Remove unnecessary columns
tokenized_datasets["train"] = tokenized_datasets["train"].remove_columns(["id", "sample_id", "tokens", "ner_tags", "ner_tags_index"])
tokenized_datasets["validation"] = tokenized_datasets["validation"].remove_columns(["id", "sample_id", "tokens", "ner_tags", "ner_tags_index"])
tokenized_datasets["test"] = tokenized_datasets["test"].remove_columns(["id", "sample_id", "tokens", "ner_tags", "ner_tags_index"])

# Set dataset format
tokenized_datasets["train"].set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_datasets["validation"].set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_datasets["test"].set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# DataLoader setup
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="pt")
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=16, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=16, collate_fn=data_collator)
test_dataloader = DataLoader(tokenized_datasets["test"], batch_size=16, collate_fn=data_collator)


# Define the model with CRF
class BertCRFNER(nn.Module):
    def __init__(self, model_name, num_labels):
        super(BertCRFNER, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(self.dropout(outputs.last_hidden_state))

        if labels is not None:
            # Replace -100 with a valid index (e.g., 0)
            valid_labels = labels.clone()
            valid_labels[labels == -100] = 0

            # Compute CRF loss
            loss = -self.crf(logits, valid_labels, mask=attention_mask.bool())
            return loss
        else:
            # Decode CRF predictions
            predictions = self.crf.decode(logits, mask=attention_mask.bool())
            return predictions

# Training configuration
num_epochs = 5
learning_rate = 3e-5
weight_decay = 0.01

# Initialize the model
improved_model = BertCRFNER(model_name=model_name, num_labels=num_labels)
improved_model.to(device)

# Optimizer and Scheduler setup
optimizer = torch.optim.AdamW(improved_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Loss tracking
train_losses = []
val_losses = []


# Modify the training setup to support checkpoint resuming
if resume_from_checkpoint:
    # Load the checkpoint
    # checkpoint = torch.load(resume_from_checkpoint)
    with gzip.open(resume_from_checkpoint, 'rb') as f:

        checkpoint = torch.load(f, map_location=device)

    # Restore model state
    improved_model.load_state_dict(checkpoint['model_state_dict'])
    improved_model.to(device)

    # Restore optimizer state
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    # Restore scheduler state
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    # Set the starting epoch and potentially modify num_epochs
    start_epoch = checkpoint['epoch']

    # Optionally prepopulate loss lists with previous losses
    train_losses = checkpoint.get('train_losses', [])
    val_losses = checkpoint.get('val_losses', [])

    print(f"Resuming training from epoch {start_epoch}")

# latest_checkpoint_path = os.path.join(google_drive_dir, "latest_checkpoint.pt")
latest_checkpoint_path = os.path.join(google_drive_dir, "latest_checkpoint.pt.gz")
# Training and Validation Loop
for epoch in range(start_epoch,num_epochs):
    # Training Phase
    improved_model.train()
    total_train_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        loss = improved_model(input_ids, attention_mask, labels=labels)
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(improved_model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    # Average training loss for the epoch
    avg_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    print(f"Epoch {epoch + 1}: Training Loss = {avg_train_loss}")

    # Validation Phase
    improved_model.eval()
    total_val_loss = 0
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            loss = improved_model(input_ids, attention_mask, labels=labels)
            total_val_loss += loss.item()

            preds = improved_model(input_ids, attention_mask)
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy().tolist())

    # Average validation loss for the epoch
    avg_val_loss = total_val_loss / len(val_dataloader)
    val_losses.append(avg_val_loss)
    print(f"Epoch {epoch + 1}: Validation Loss = {avg_val_loss}")

    # torch.save({
    #     'epoch': epoch + 1,
    #     'model_state_dict': improved_model.state_dict(),
    #     'optimizer_state_dict': optimizer.state_dict(),
    #     'scheduler_state_dict': scheduler.state_dict()
    # }, latest_checkpoint_path)

    with gzip.open(latest_checkpoint_path, 'wb') as f:
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': improved_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict()
        }, f)


    print(f"Checkpoint saved at: {latest_checkpoint_path}")

    # Decode and print classification report
    decoded_predictions = []
    decoded_labels = []
    for preds, labels in zip(predictions, true_labels):
        valid_preds = [p for p, l in zip(preds, labels) if l != -100]
        valid_labels = [l for l in labels if l != -100]
        decoded_predictions.append([label_list[p] for p in valid_preds])
        decoded_labels.append([label_list[l] for l in valid_labels])

    print(f"Classification Report for Epoch {epoch + 1}:")
    print(classification_report(decoded_labels, decoded_predictions))

# Plot loss curves
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss', marker='o')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', marker='o')
plt.title('Training and Validation Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
loss_curve_path = os.path.join(google_drive_dir, 'loss_curve.png')
plt.savefig(loss_curve_path)
plt.close()

print("Training completed. Checkpoints and loss curve saved in /content/checkpoints/")

Mounted at /content/drive
Using device: cuda
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
README.md: 100%
 4.45k/4.45k [00:00<00:00, 295kB/s]
multiconer_v2.py: 100%
 13.1k/13.1k [00:00<00:00, 968kB/s]
0000.parquet: 100%
 1.98M/1.98M [00:00<00:00, 35.9MB/s]
0000.parquet: 100%
 117k/117k [00:00<00:00, 6.63MB/s]
0000.parquet: 100%
 29.0M/29.0M [00:00<00:00, 196MB/s]
Generating train split: 100%
 16778/16778 [00:00<00:00, 146752.40 examples/s]
Generating validation split: 100%
 871/871 [00:00<00:00, 22792.85 examples/s]
Generating test split: 100%
 249980/249980 [00:00<00:00, 268050.39 examples/s]
tokenizer_config.json: 100%
 49.0/49.0 [00:00<00:00, 3.13kB/s]
config.json: 100%
 762/762 [00:00<00:00, 49.2kB/s]
vocab.txt: 100%
 213k/213k [00:00<00:00, 449kB/s]
tokenizer.json: 100%
 436k/436k [00:00<00:00, 29.1MB/s]
Map: 100%
 16778/16778 [00:05<00:00, 2146.63 examples/s]
Map: 100%
 871/871 [00:00<00:00, 2321.71 examples/s]
Map: 100%
 249980/249980 [01:16<00:00, 3728.98 examples/s]
model.safetensors: 100%
 1.34G/1.34G [00:08<00:00, 226MB/s]
Training Epoch 1: 100%|██████████| 1049/1049 [22:37<00:00,  1.29s/it]
Epoch 1: Training Loss = 117.65626361508728
Evaluating: 100%|██████████| 55/55 [00:47<00:00,  1.17it/s]
Epoch 1: Validation Loss = 65.69587013938211
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Checkpoint saved at: /content/drive/My Drive/NER_checkpoints/latest_checkpoint.pt.gz
Classification Report for Epoch 1:
                       precision    recall  f1-score   support

AerospaceManufacturer       0.38      0.60      0.46        10
  AnatomicalStructure       0.65      0.76      0.70        17
              ArtWork       0.50      0.38      0.43        13
               Artist       0.74      0.83      0.78       212
              Athlete       0.68      0.77      0.72        79
      CarManufacturer       0.20      0.15      0.17        13
               Cleric       0.45      0.33      0.38        15
             Clothing       0.67      0.60      0.63        10
              Disease       0.46      0.33      0.39        18
                Drink       0.27      0.27      0.27        11
             Facility       0.51      0.67      0.58        52
                 Food       0.46      0.32      0.37        19
      HumanSettlement       0.78      0.81      0.79       109
     MedicalProcedure       0.40      0.62      0.48        13
   Medication/Vaccine       0.68      0.72      0.70        18
           MusicalGRP       0.66      0.62      0.64        37
          MusicalWork       0.73      0.61      0.66        61
                  ORG       0.43      0.54      0.48        78
             OtherLOC       0.38      0.19      0.25        16
             OtherPER       0.33      0.33      0.33        91
            OtherPROD       0.38      0.39      0.38        49
           Politician       0.39      0.53      0.45        53
          PrivateCorp       0.00      0.00      0.00        11
           PublicCorp       0.39      0.57      0.46        28
            Scientist       0.29      0.33      0.31        15
             Software       0.52      0.46      0.49        26
            SportsGRP       0.86      0.78      0.82        41
        SportsManager       0.69      0.69      0.69        16
              Station       0.73      0.80      0.76        20
              Symptom       0.33      0.20      0.25        10
              Vehicle       0.39      0.35      0.37        20
           VisualWork       0.75      0.75      0.75        61
          WrittenWork       0.60      0.63      0.61        54

            micro avg       0.58      0.61      0.60      1296
            macro avg       0.51      0.51      0.50      1296
         weighted avg       0.58      0.61      0.59      1296

Training Epoch 2: 100%|██████████| 1049/1049 [22:35<00:00,  1.29s/it]
Epoch 2: Training Loss = 53.443223507774796
Evaluating: 100%|██████████| 55/55 [00:46<00:00,  1.17it/s]
Epoch 2: Validation Loss = 60.406124184348364
Checkpoint saved at: /content/drive/My Drive/NER_checkpoints/latest_checkpoint.pt.gz
Classification Report for Epoch 2:
                       precision    recall  f1-score   support

AerospaceManufacturer       0.50      0.60      0.55        10
  AnatomicalStructure       0.69      0.65      0.67        17
              ArtWork       0.70      0.54      0.61        13
               Artist       0.78      0.83      0.80       212
              Athlete       0.70      0.80      0.75        79
      CarManufacturer       0.47      0.54      0.50        13
               Cleric       0.45      0.33      0.38        15
             Clothing       0.45      0.50      0.48        10
              Disease       0.44      0.44      0.44        18
                Drink       0.50      0.55      0.52        11
             Facility       0.68      0.69      0.69        52
                 Food       0.64      0.47      0.55        19
      HumanSettlement       0.82      0.89      0.85       109
     MedicalProcedure       0.64      0.69      0.67        13
   Medication/Vaccine       0.68      0.83      0.75        18
           MusicalGRP       0.73      0.81      0.77        37
          MusicalWork       0.74      0.74      0.74        61
                  ORG       0.53      0.51      0.52        78
             OtherLOC       0.78      0.44      0.56        16
             OtherPER       0.45      0.49      0.47        91
            OtherPROD       0.38      0.33      0.35        49
           Politician       0.52      0.53      0.52        53
          PrivateCorp       0.33      0.09      0.14        11
           PublicCorp       0.44      0.50      0.47        28
            Scientist       0.50      0.40      0.44        15
             Software       0.68      0.50      0.58        26
            SportsGRP       0.84      0.90      0.87        41
        SportsManager       0.82      0.56      0.67        16
              Station       0.94      0.75      0.83        20
              Symptom       0.00      0.00      0.00        10
              Vehicle       0.53      0.45      0.49        20
           VisualWork       0.66      0.69      0.67        61
          WrittenWork       0.67      0.72      0.70        54

            micro avg       0.66      0.66      0.66      1296
            macro avg       0.60      0.57      0.58      1296
         weighted avg       0.65      0.66      0.65      1296

Training Epoch 3: 100%|██████████| 1049/1049 [22:39<00:00,  1.30s/it]
Epoch 3: Training Loss = 33.29585610126972
Evaluating: 100%|██████████| 55/55 [00:47<00:00,  1.17it/s]
Epoch 3: Validation Loss = 60.80253365256569
Checkpoint saved at: /content/drive/My Drive/NER_checkpoints/latest_checkpoint.pt.gz
Classification Report for Epoch 3:
                       precision    recall  f1-score   support

AerospaceManufacturer       0.57      0.80      0.67        10
  AnatomicalStructure       0.69      0.65      0.67        17
              ArtWork       0.50      0.23      0.32        13
               Artist       0.73      0.87      0.80       212
              Athlete       0.76      0.78      0.77        79
      CarManufacturer       0.54      0.54      0.54        13
               Cleric       0.50      0.33      0.40        15
             Clothing       0.64      0.70      0.67        10
              Disease       0.50      0.61      0.55        18
                Drink       0.60      0.82      0.69        11
             Facility       0.64      0.79      0.71        52
                 Food       0.83      0.53      0.65        19
      HumanSettlement       0.80      0.90      0.85       109
     MedicalProcedure       0.57      0.62      0.59        13
   Medication/Vaccine       0.71      0.83      0.77        18
           MusicalGRP       0.86      0.81      0.83        37
          MusicalWork       0.69      0.77      0.73        61
                  ORG       0.63      0.54      0.58        78
             OtherLOC       0.62      0.31      0.42        16
             OtherPER       0.51      0.44      0.47        91
            OtherPROD       0.50      0.49      0.49        49
           Politician       0.52      0.57      0.54        53
          PrivateCorp       0.20      0.09      0.13        11
           PublicCorp       0.59      0.57      0.58        28
            Scientist       0.43      0.40      0.41        15
             Software       0.64      0.62      0.63        26
            SportsGRP       0.86      0.93      0.89        41
        SportsManager       0.85      0.69      0.76        16
              Station       0.80      0.80      0.80        20
              Symptom       0.60      0.30      0.40        10
              Vehicle       0.56      0.50      0.53        20
           VisualWork       0.62      0.77      0.69        61
          WrittenWork       0.74      0.69      0.71        54

            micro avg       0.67      0.69      0.68      1296
            macro avg       0.63      0.61      0.61      1296
         weighted avg       0.67      0.69      0.67      1296

Training Epoch 4: 100%|██████████| 1049/1049 [22:35<00:00,  1.29s/it]
Epoch 4: Training Loss = 21.13793125279866
Evaluating: 100%|██████████| 55/55 [00:47<00:00,  1.17it/s]
Epoch 4: Validation Loss = 69.80272369384765
Checkpoint saved at: /content/drive/My Drive/NER_checkpoints/latest_checkpoint.pt.gz
Classification Report for Epoch 4:
                       precision    recall  f1-score   support

AerospaceManufacturer       0.62      0.80      0.70        10
  AnatomicalStructure       0.67      0.71      0.69        17
              ArtWork       0.46      0.46      0.46        13
               Artist       0.81      0.79      0.80       212
              Athlete       0.81      0.75      0.78        79
      CarManufacturer       0.62      0.62      0.62        13
               Cleric       0.50      0.40      0.44        15
             Clothing       0.67      0.80      0.73        10
              Disease       0.56      0.50      0.53        18
                Drink       0.58      0.64      0.61        11
             Facility       0.67      0.79      0.73        52
                 Food       0.50      0.58      0.54        19
      HumanSettlement       0.81      0.87      0.84       109
     MedicalProcedure       0.57      0.62      0.59        13
   Medication/Vaccine       0.71      0.83      0.77        18
           MusicalGRP       0.86      0.81      0.83        37
          MusicalWork       0.76      0.82      0.79        61
                  ORG       0.58      0.58      0.58        78
             OtherLOC       0.42      0.31      0.36        16
             OtherPER       0.47      0.68      0.56        91
            OtherPROD       0.43      0.47      0.45        49
           Politician       0.59      0.51      0.55        53
          PrivateCorp       0.33      0.18      0.24        11
           PublicCorp       0.58      0.54      0.56        28
            Scientist       0.36      0.33      0.34        15
             Software       0.62      0.50      0.55        26
            SportsGRP       0.80      0.90      0.85        41
        SportsManager       0.67      0.62      0.65        16
              Station       0.83      0.75      0.79        20
              Symptom       0.67      0.60      0.63        10
              Vehicle       0.48      0.50      0.49        20
           VisualWork       0.74      0.75      0.75        61
          WrittenWork       0.63      0.70      0.67        54

            micro avg       0.67      0.69      0.68      1296
            macro avg       0.62      0.63      0.62      1296
         weighted avg       0.67      0.69      0.68      1296

In [ ]:
# Reduce the test dataset size (e.g., 10% of the original size)
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(int(len(dataset["test"]) * 0.1)))

# Reapply tokenization to the reduced test dataset
tokenized_datasets["test"] = dataset["test"].map(tokenize_and_align_labels_with_crf, batched=True)
tokenized_datasets["test"] = tokenized_datasets["test"].remove_columns(["id", "sample_id", "tokens", "ner_tags", "ner_tags_index"])
tokenized_datasets["test"].set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Recreate the test dataloader with the smaller dataset
test_dataloader = DataLoader(tokenized_datasets["test"], batch_size=16, collate_fn=data_collator)

# Set the model to evaluation mode
improved_model.eval()

# Lists to store predictions and true labels
all_predictions = []
all_true_labels = []

# Disable gradient computation during testing
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Get predictions
        preds = improved_model(input_ids, attention_mask)

        # Extend predictions and true labels
        all_predictions.extend(preds)
        all_true_labels.extend(labels.cpu().numpy().tolist())

# Decode predictions and true labels
decoded_predictions = []
decoded_labels = []
for preds, labels in zip(all_predictions, all_true_labels):
    valid_preds = [p for p, l in zip(preds, labels) if l != -100]
    valid_labels = [l for l in labels if l != -100]
    decoded_predictions.append([label_list[p] for p in valid_preds])
    decoded_labels.append([label_list[l] for l in valid_labels])

# Print classification report
print("Classification Report on Test Data (10% subset):")
print(classification_report(decoded_labels, decoded_predictions))

Map: 100%
 24998/24998 [00:09<00:00, 3808.53 examples/s]
Testing: 100%|██████████| 1563/1563 [11:00<00:00,  2.37it/s]
Classification Report on Test Data (10% subset):
                       precision    recall  f1-score   support

AerospaceManufacturer       0.35      0.63      0.45        82
  AnatomicalStructure       0.65      0.71      0.68       567
              ArtWork       0.40      0.56      0.46       149
               Artist       0.71      0.81      0.76      5696
              Athlete       0.77      0.76      0.77      2676
      CarManufacturer       0.58      0.60      0.59       297
               Cleric       0.50      0.49      0.50       489
             Clothing       0.58      0.61      0.60       224
              Disease       0.65      0.63      0.64       566
                Drink       0.52      0.55      0.53       203
             Facility       0.62      0.67      0.65      1605
                 Food       0.50      0.52      0.51       545
      HumanSettlement       0.86      0.87      0.87      4205
     MedicalProcedure       0.60      0.59      0.59       383
   Medication/Vaccine       0.68      0.71      0.69       586
           MusicalGRP       0.63      0.67      0.65      1248
          MusicalWork       0.66      0.72      0.69      1525
                  ORG       0.54      0.62      0.58      2307
             OtherLOC       0.48      0.48      0.48       472
             OtherPER       0.38      0.44      0.41      2170
            OtherPROD       0.46      0.48      0.47      1165
           Politician       0.52      0.53      0.53      1626
          PrivateCorp       0.17      0.32      0.22       111
           PublicCorp       0.54      0.53      0.53       664
            Scientist       0.44      0.34      0.38       482
             Software       0.65      0.69      0.67       883
            SportsGRP       0.78      0.82      0.80      1280
        SportsManager       0.53      0.53      0.53       509
              Station       0.77      0.78      0.77       617
              Symptom       0.38      0.52      0.44       183
              Vehicle       0.46      0.51      0.49       618
           VisualWork       0.68      0.73      0.71      2000
          WrittenWork       0.65      0.66      0.66      1662

            micro avg       0.64      0.68      0.66     37795
            macro avg       0.57      0.61      0.58     37795
         weighted avg       0.64      0.68      0.66     37795